# <font color='black'>НИС: регрессионный анализ, 2025 </font>
## <font color='black'> Практическое занятие: Мультиколлинеарность. VIF </font>

## <font color='black'>Модель множественной линейной регрессии. Мультиколлинеарность </font>
На этом занятии мы продолжим работать с данными из статьи [Kalenborn C., Lessman C., 2013](https://yadi.sk/i/nlEQUoWKiqY0UA). Одна из частей анализа в данной статье выполнена на основе cross-section data (использованы усредненные данные за 2005 - 2010 гг.). Авторы изучают взаимосвязь уровня коррупции и демократии, предполагая, что ее характер зависит от значений показателя свободы прессы. Кратко о данных:
* cpi - уровень коррупции: Corruption Perception Index. Непрерывная шкала от 0 до 10, где 10 означает наиболее высокий уровень коррупции.
* dem - индекс демократии: Vanhanen’s democratization index. Непрерывная шкала от 0 до 100, где 100 означает максимальное значение уровня демократии.
* fp - свобода прессы: Freedom House. Приведен к непрерывной шкале от 0 до 100, где 100 - наиболее высокое значение свободы прессы.
* loggdppc - натуральный логарифм ВВП на душу населения. World Bank.
* stab - уровень политической стабильности. Индекс построен на основе показателей "Political Stability" и "Absence of Violence/Terrorism" из the Worldwide Governance Indicators. Непрерывная шкала от -2.5 до 2.5, где 2.5 соответствует наиболее высокому уровню политической стабильности.
* britcol - дамми-переменная, где 1 - бывшая британская колония.

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
import seaborn as sns
import numpy as np
import matplotlib.pyplot as mpl
%matplotlib inline

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

Откроем массив данных:

In [ ]:
lab4 = pd.read_stata('lab1.dta')
lab4 = lab4.dropna()

И оценим линейную множественную регрессионную модель с уже знакомой Вам спецификацией:

In [ ]:
m1 = smf.ols(formula = "cpi ~ dem + fp + loggdppc + britcol + stab", data = lab4).fit(cov_type="HC3")
print(m1.summary())

С точки зрения такого сюжета, как мультиколлинеарность, полезно будет посмотреть на то, насколько сильно связаны объясняющие переменные в модели. Выведем набор графиков, включающий и визуализацию распределений самих переменных (по главной диагонали), и диаграммы рассеяния.

In [ ]:
sns.pairplot(lab4[["dem", "fp", "stab", "loggdppc", "britcol"]])

Также ниже представим корреляционную матрицу для предикторов:

In [ ]:
lab4[["dem", "fp", "stab", "loggdppc", "britcol"]].corr().round(3)

Рассчитаем показатели variance inflation factor:

In [ ]:
X = lab4[["dem", "fp", "stab", "loggdppc", "britcol"]]
X = add_constant(X)

vif_data = pd.DataFrame({'variables':X.columns[1:], 'VIF':[variance_inflation_factor(X.values, i+1) for i in range(len(X.columns[1:]))]})
print(vif_data)

Также для тренировки самостоятельно посчитаем VIF для показателя dem, построив предварительно соответствующую вспомогательную модель и выгрузив ее коэффициент детерминации:

In [ ]:
m1_aux = smf.ols(formula = "dem ~ fp + loggdppc + britcol + stab", data = lab4).fit()

vif_dem = 1/(1-m1_aux.rsquared)
print(vif_dem)

Для лучшего понимания, как связаны $R^2$ из вспомогательной регрессионной модели и значения VIF, построим график:

In [ ]:
rsquared = np.linspace(0, 0.99)
VIF = 1/(1-rsquared)
fig, axes = mpl.subplots()
mpl.plot(rsquared, VIF, 'b')
axes.set_xlabel('$R^2$')
axes.set_ylabel('VIF')
mpl.title('VIF как функция от $R^2$')
mpl.show()

Рассмотрим, что можно было бы сделать, если бы все-таки сильная мультиколлинеарность была обнаружена. Мы уже обсуждали, что в этом случае нам может помочь метод главных компонент. На основе исходных данных можно построить новые индексы - линейные комбинации стандартизированных показателей (объясняющих переменных). Главные компоненты будут нескоррелированны между собой.

Возьмем за основу для тренировки такие достаточно сильно скоррелированные между собой предикторы, как dem, fp, stab, loggdppc.

И для начала стандартизируем наши данные.

In [ ]:
pca_dta = lab4[["dem", "fp", "stab", "loggdppc"]]
pca_dta_stand =  StandardScaler().fit_transform(pca_dta)
pca_dta_stand = pd.DataFrame(pca_dta_stand, columns=pca_dta.columns)
pca_dta_stand.head(5)

Реализуем процедуру МГК и выведем значения информативности решения (сами значения вариаций, и для удобства долю общей вариации, объясняемую главными компонентами)

In [ ]:
pca_results = PCA().fit(pca_dta_stand)

print(pca_results.explained_variance_)
print(pca_results.explained_variance_ratio_)

Визуализируем информативность решения на графике так называемой "каменистой осыпи".

In [ ]:
components = range(1, len(pca_results.explained_variance_ratio_) + 1)

mpl.figure(figsize=(8, 5))
mpl.plot(components, pca_results.explained_variance_ratio_, 'bo-', linewidth=2)
mpl.bar(components, pca_results.explained_variance_ratio_, alpha=0.3)
mpl.xlabel('Главные компоненты')
mpl.ylabel('Доля объясненной дисперсии')
mpl.title('График каменистой осыпи')
mpl.xticks(components)
mpl.grid(True, alpha=0.3)
mpl.show()

Представим отдельно веса, с которыми исходные стандартизированные показатели учитываются в главных компонентах:

In [ ]:
loadings = pca_results.components_
pca_list = ["PC"+str(i+1) for i in range(len(pca_dta_stand.columns[0:]))]
loadings = pd.DataFrame.from_dict(dict(zip(pca_list, loadings)))
loadings['variable'] = pca_dta_stand.columns.values
loadings = loadings.set_index('variable')
print(loadings)

Если ГК интерпретируемые, Вы можете включить их вместо исходных показателей в регрессионную модель. Можно включить только первые несколько ГК (количество определяется на основе критерия Кайзера и графика каменистой осыпи, как мы уже обсудили), объясняющие наибольшую долю дисперсии исходных данных. Ниже покажем, как можно сохранить полученные ГК в исходном массиве данных:

In [ ]:
PC = pca_results.transform(pca_dta_stand)
lab4["PC1"] = PC[:,0]
lab4.head()
